In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
datosFull = np.loadtxt('data_to_fit.txt')
X = datosFull[:,0]
Y = datosFull[:,1]
SigmaY = datosFull[:,2]

In [ ]:
def model_A(x,params):
    y = params[0] + params[1]*x + params[2]*x**2
    return y

def model_B(x,params):
    y = params[0]*np.exp(-0.5*(x-params[1])**2/params[2]**2)
    return y

In [ ]:
def log_prior(w):
    return 0

def log_verosimilitud_modelo(betas,X,Y,Sigma,modelo):
    delta = modelo(X,betas) - Y
    chi2 = (delta/Sigma)**2
    return np.sum(-chi2/2)

In [ ]:
def MCMC(X,Y,Sigma,num_coefs,N,modelo):
    betas = [np.zeros(num_coefs)]
    for i in range(0,N):
        paso = np.random.normal(loc=0.0,scale=1,size=len(betas[i]))
        nuevos_betas = betas[i]+paso
        log_nuevo = log_verosimilitud_modelo(nuevos_betas,X,Y,Sigma,modelo) + log_prior(nuevos_betas)
        log_viejo = log_verosimilitud_modelo(betas[i],X,Y,Sigma,modelo) + log_prior(betas[i])
        r = min(0,log_nuevo-log_viejo)
        alfa = np.random.rand()
        if np.exp(r)>=alfa:
            betas.append(nuevos_betas)
        else:
            betas.append(betas[i])
            
    return np.array(betas[int(N/2):])

In [ ]:
betasB = MCMC(X,Y,SigmaY,3,20000,model_B).mean(axis=0)
betasA = MCMC(X,Y,SigmaY,3,20000,model_A).mean(axis=0)

In [ ]:
x_test = np.linspace(3,6.5)
plt.figure()
plt.scatter(X,Y)
plt.plot(x_test,model_B(x_test,betasB),c='b')
plt.plot(x_test,model_A(x_test,betasA),c='r')

In [ ]:
log_MB = log_verosimilitud_modelo(betasB,X,Y,SigmaY,model_B)
log_MA = log_verosimilitud_modelo(betasA,X,Y,SigmaY,model_A)
BayesFactor = np.sum(np.exp(log_MB)*(log_MB-log_MA))

print(BayesFactor)
print(np.sum(np.exp(log_MA)*(log_MA-log_MB)))